## Confirm rotation of minimal-basis S<sub>N</sub>2 reaction

In [ ]:
from collections import defaultdict

import json

import numpy as np

import plotly.express as px

from ase.data import atomic_numbers
from ase.visualize import view

from pymatgen.core.structure import Molecule
from pymatgen.io.ase import AseAtomsAdaptor

import basis_set_exchange as bse

import torch

from e3nn import o3

from minimal_basis.predata.matrices import ReducedBasisMatrices 
from minimal_basis.transforms.rotations import RotationMatrix

from instance_mongodb import instance_mongodb_sei

In [ ]:
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_sn2_reaction_calculation

data = defaultdict(list)

basis_set = '6-31g*'
basis_info = bse.get_basis(basis_set, fmt='json', elements=list(range(1, 18)))
basis_info = json.loads(basis_info)

electron_shells = {k: basis_info['elements'][k]['electron_shells'] for k in basis_info['elements'].keys()}
angular_momenta = {k: [shell['angular_momentum'] for shell in electron_shells[k]] for k in electron_shells.keys()}
angular_momenta = {k: [item for sublist in angular_momenta[k] for item in sublist] for k in angular_momenta.keys()}

for doc in collection.find({'tags.state': 'transition_state'}).sort('tags.idx', 1):

    _alpha_coeff_matrix = doc['calcs_reversed'][0]['alpha_coeff_matrix']
    _alpha_eigenvalues = doc['calcs_reversed'][0]['alpha_eigenvalues']
    _alpha_fock_matrix = doc['calcs_reversed'][0]['alpha_fock_matrix']

    _alpha_coeff_matrix = np.array(_alpha_coeff_matrix)
    _alpha_eigenvalues = np.array(_alpha_eigenvalues)
    _alpha_fock_matrix = np.array(_alpha_fock_matrix)

    molecule = Molecule.from_dict(doc['orig']['molecule'])
    symbols = [site.specie.symbol for site in molecule.sites]
    atom_numbers = [atomic_numbers[symbol] for symbol in symbols]

    _irreps = ""
    indices_to_keep = []
    _idx = 0
    _basis_functions_orbital = []
    for atom_number in atom_numbers:
        _angular_momenta = angular_momenta[str(atom_number)]
        _angular_momenta = np.array(_angular_momenta)
        _basis_functions = 2*_angular_momenta + 1
        for _basis_function in _basis_functions:
            if _basis_function == 1:
                _basis_functions_orbital.extend(['s'])
                _irreps += "+1x0e"
                indices_to_keep.append(_idx)
                _idx += 1
            elif _basis_function == 3:
                _basis_functions_orbital.extend(['p', 'p', 'p'])
                _irreps += "+1x1o"
                indices_to_keep.extend([_idx, _idx+1, _idx+2])
                _idx += 3
            elif _basis_function == 5 and basis_set != '6-31g*':
                _idx += 5
            elif _basis_function == 5 and basis_set == '6-31g*':
                _idx += 6

    _irreps = _irreps[1:]

    base_quantities_qchem = ReducedBasisMatrices(
        fock_matrix=_alpha_fock_matrix,
        eigenvalues=_alpha_eigenvalues,
        coeff_matrix=_alpha_coeff_matrix,
        indices_to_keep=indices_to_keep,
    )

    alpha_ortho_coeff_matrix = base_quantities_qchem.get_ortho_coeff_matrix()
    alpha_coeff_matrix = base_quantities_qchem.get_coeff_matrix()
    alpha_fock_matrix = base_quantities_qchem.get_fock_matrix()
    alpha_eigenvalues = base_quantities_qchem.eigenval_ortho_fock

    data['alpha_coeff_matrix'].append(alpha_coeff_matrix)
    data['alpha_eigenvalues'].append(alpha_eigenvalues)
    data['alpha_fock_matrix'].append(alpha_fock_matrix)
    data['alpha_ortho_coeff_matrix'].append(alpha_ortho_coeff_matrix)
    data['basis_functions_orbital'].append(_basis_functions_orbital)
    data['irreps'].append(_irreps)
    data['euler_angles'].append(doc['tags']['euler_angles'])
    data['idx'].append(doc['tags']['idx'])
    data['molecule'].append(molecule)

for key in data.keys():
    if key != 'molecule':
        data[key] = np.array(data[key])

In [ ]:
quantity = "alpha_ortho_coeff_matrix"

In [ ]:
all_molecules = data['molecule']
all_atoms = [AseAtomsAdaptor.get_atoms(molecule) for molecule in all_molecules]
view(all_atoms[7], viewer="x3d")

In [ ]:
fig = px.imshow(data[quantity], animation_frame=0, range_color=[-1, 1])
labels = data['basis_functions_orbital'][0]
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Coefficient matrix from DFT calculation")
fig.show()

In [ ]:
new_coeff_matrix = []
calculated_coeff_matrix = data[quantity]
original_coeff_matrix = calculated_coeff_matrix[0] 

for _idx, angles in enumerate(data['euler_angles']):

    torch_angles = torch.tensor(angles, dtype=torch.float64)

    irreps = data['irreps'][_idx]
    irreps = o3.Irreps(irreps)

    rotation_matrix = RotationMatrix(angle_type="euler", angles=angles)() 
    rotation_matrix = torch.tensor(rotation_matrix, dtype=torch.float64)
    
    if _idx == 0:
        rotation_matrix_0 = rotation_matrix
    
    rotation_matrix = rotation_matrix @ rotation_matrix_0.T
     
    D_matrix = irreps.D_from_matrix(rotation_matrix)
    D_matrix = D_matrix.detach().numpy()

    _new_coeff_matrix = np.zeros_like(original_coeff_matrix)
    for i in range(original_coeff_matrix.shape[1]):
        _new_coeff_matrix[:, i] = original_coeff_matrix[:, i] @ D_matrix.T

    new_coeff_matrix.append(_new_coeff_matrix)

new_coeff_matrix = np.array(new_coeff_matrix)

fig = px.imshow(new_coeff_matrix, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Rotated coefficient matrix from D-matrix based on rotated Euler angles")
fig.show()

In [ ]:
difference_matrices = np.abs(new_coeff_matrix) - np.abs(calculated_coeff_matrix)
fig = px.imshow(difference_matrices, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Difference between rotated and calculated coefficient matrix")
fig.show()